In [11]:
from qiskit_nature.second_quantization.drivers import PySCFDriver
from qiskit_nature.second_quantization.problems import ElectronicStructureProblem
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper
from qiskit_nature.algorithms import GroundStateEigensolver
from qiskit.algorithms import NumPyMinimumEigensolver

# 1. Set up the molecule using PySCF driver via Qiskit
driver = PySCFDriver(atom='H 0 0 0; H 0 0 0.735', basis='sto3g')
es_problem = ElectronicStructureProblem(driver)

# 2. Build second-quantized Hamiltonian
second_q_ops = es_problem.second_q_ops()
electronic_hamiltonian = second_q_ops[0]  # The first operator is the Hamiltonian

# 3. Map to qubit Hamiltonian using Jordan-Wigner
qubit_converter = QubitConverter(mapper=JordanWignerMapper())
qubit_op = qubit_converter.convert(electronic_hamiltonian, num_particles=es_problem.num_particles)

# 4. Print Pauli terms and coefficients
for pauli_term in qubit_op.to_list():
    print(pauli_term)


ModuleNotFoundError: No module named 'qiskit_nature'

# CAFQA

In [1]:
import sys
sys.path.append("../")
from clapton.clapton import claptonize, claptonize_opt, claptonize_opt2
from clapton.ansatzes import circular_ansatz
import numpy as np

d:\research-impl\clapton\clapton\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# define Hamiltonian, e.g. 3q Heisenberg model with random coefficients
paulis = ["XXI", "IXX", "YYI", "IYY", "ZZI", "IZZ"]
np.random.seed(42)  # You can replace 42 with any integer seed
coeffs = np.random.random(len(paulis))
# coeffs = np.ones((len(paulis)))
print(coeffs)

[0.37454012 0.95071431 0.73199394 0.59865848 0.15601864 0.15599452]


In [3]:
# define parametrized Clifford circuit that is being optimized over
# here we use the circular_ansatz template
# we fix 2q gates as they will not be optimized over
vqe_pcirc = circular_ansatz(N=len(paulis[0]), reps=1, fix_2q=True)

In [4]:
# the circuit consists of parametrized gates
for gate in vqe_pcirc.gates:
    print(gate.label, gate.is_fixed())

RY False
RY False
RY False
RZ False
RZ False
RZ False
2Q True
2Q True
2Q True
RY False
RY False
RY False
RZ False
RZ False
RZ False


In [5]:
# non-fixed gates will be optimized over
# RY and RZ gates can assume 4 values k = 0,1,2,3 which descripe multiples of pi/2

In [6]:
# the initial parameters are all 0
vqe_pcirc.read()

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [7]:
# we can look at the corresponding stim circuit
vqe_pcirc.stim_circuit().diagram()

q0: -I-I-X-@-I-I---
         | |
q1: -I-I-|-X-@-I-I-
         |   |
q2: -I-I-@---X-I-I-

In [8]:
# we can assign a different set of parameters
vqe_pcirc.assign([0,1,2,3,0,1,2,3,0,1,2,3])

In [9]:
vqe_pcirc.stim_circuit().diagram()

q0: -I------S_DAG-X-@-Y-S----------------
                  | |
q1: -SQRT_Y-I-----|-X-@-SQRT_Y_DAG-Z-----
                  |   |
q2: -Y------S-----@---X-I----------S_DAG-

In [10]:
# we can perform CAFQA by using the main optimization function "claptonize"
ks_best, _, energy_best = claptonize_opt2(
    paulis,
    coeffs,
    vqe_pcirc,
    n_proc=4,           # total number of processes in parallel
    n_starts=4,         # number of random genetic algorithm starts in parallel
    n_rounds=1,         # number of budget rounds, if None it will terminate itself
    callback=print,     # callback for internal parameter (#iteration, energies, ks) processing
    budget=20           # budget per genetic algorithm instance
)

[I 2025-07-17 22:38:56,615] A new study created in memory with name: no-name-0bc7c60b-d2f4-4230-8b47-0424f1a0c3e8
[I 2025-07-17 22:38:56,625] Trial 0 finished with value: 0.0 and parameters: {'x0': 1, 'x1': 3, 'x2': 2, 'x3': 2, 'x4': 0, 'x5': 0, 'x6': 0, 'x7': 3, 'x8': 2, 'x9': 2, 'x10': 0, 'x11': 3}. Best is trial 0 with value: 0.0.
[I 2025-07-17 22:38:56,634] Trial 1 finished with value: 0.0 and parameters: {'x0': 3, 'x1': 0, 'x2': 0, 'x3': 0, 'x4': 1, 'x5': 2, 'x6': 1, 'x7': 1, 'x8': 2, 'x9': 0, 'x10': 1, 'x11': 1}. Best is trial 0 with value: 0.0.
[I 2025-07-17 22:38:56,645] Trial 2 finished with value: -0.39212260375335384 and parameters: {'x0': 1, 'x1': 3, 'x2': 0, 'x3': 2, 'x4': 2, 'x5': 0, 'x6': 2, 'x7': 0, 'x8': 0, 'x9': 3, 'x10': 3, 'x11': 3}. Best is trial 2 with value: -0.39212260375335384.
[I 2025-07-17 22:38:56,655] Trial 3 finished with value: 0.0 and parameters: {'x0': 1, 'x1': 0, 'x2': 2, 'x3': 1, 'x4': 0, 'x5': 1, 'x6': 0, 'x7': 3, 'x8': 1, 'x9': 2, 'x10': 1, 'x11': 2

HEREEEEE4
[2, 3, 0, 2, 1, 2, 2, 2, 0, 3, 1, 0]


In [11]:
# the best parameters are
ks_best

[2, 3, 0, 2, 1, 2, 2, 2, 0, 3, 1, 0]

In [ ]:
# with energy
energy_best

np.float64(-1.7053673109431555)

In [13]:
# the corresponding circuit is
vqe_pcirc.assign(ks_best)
vqe_pcirc.stim_circuit().diagram()

q0: -Y----------Z-X-@-Y-S_DAG---
                  | |
q1: -SQRT_Y_DAG-S-|-X-@-Y-----S-
                  |   |
q2: -I----------Z-@---X-I-----I-

# noisy CAFQA (nCAFQA)

In [ ]:
from clapton.depolarization import GateGeneralDepolarizationModel

In [ ]:
# let's add a noise model where we specify global 1q and 2q gate errors
nm = GateGeneralDepolarizationModel(p1=0.005, p2=0.02)

In [ ]:
vqe_pcirc = circular_ansatz(N=len(paulis[0]), reps=1, fix_2q=True)
vqe_pcirc.add_depolarization_model(nm)

In [ ]:
# after every gate a depol channel is added
vqe_pcirc.stim_circuit().diagram()

q0: -I-DEPOLARIZE1(0.005)--------------------I------------------DEPOLARIZE1(0.005)---------------------------------------X-DEPOLARIZE2(0.02)-@-DEPOLARIZE2(0.02)---I-----------------DEPOLARIZE1(0.005)--------------------I------------------DEPOLARIZE1(0.005)---------------------------------------
                                                                                                                         | |                 | |
q1: ---I------------------DEPOLARIZE1(0.005)--------------------I------------------DEPOLARIZE1(0.005)--------------------|-|-----------------X-DEPOLARIZE2(0.02)-@-DEPOLARIZE2(0.02)-I------------------DEPOLARIZE1(0.005)--------------------I------------------DEPOLARIZE1(0.005)--------------------
                                                                                                                         | |                                     | |
q2: ----------------------I------------------DEPOLARIZE1(0.005)--------------------I------------------DEPOLARIZE1(0.005)-@-DEPOLARIZE2(0.02)---------------------X-DEPOLARIZE2(0.02)--------------------I------------------DEPOLARIZE1(0.005)--------------------I------------------DEPOLARIZE1(0.005)-

In [ ]:
# we can perform nCAFQA by using the main optimization function "claptonize"
# now with the noisy circuit
# this is slower, as the noisy circuit needs to be sampled
ks_best, energy_noisy, energy_noiseless = claptonize(
    paulis,
    coeffs,
    vqe_pcirc,
    n_proc=4,           # total number of processes in parallel
    n_starts=4,         # number of random genetic algorithm starts in parallel
    n_rounds=1,         # number of budget rounds, if None it will terminate itself
    callback=print,     # callback for internal parameter (#iteration, energies, ks) processing
    budget=20           # budget per genetic algorithm instance
)

STARTING ROUND 0


started GA at id 1 with 1 procs
started GA at id 2 with 1 procs




/local/lmseifert/conda_envs/clapton/lib/python3.12/site-packages/pygad/pygad.py:1139: UserWarning: The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.
  warnings.warn("The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.")


started GA at id None with 1 procs

started GA at id 3 with 1 procs


/local/lmseifert/conda_envs/clapton/lib/python3.12/site-packages/pygad/pygad.py:1139: UserWarning: The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.
  warnings.warn("The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.")
/local/lmseifert/conda_envs/clapton/lib/python3.12/site-packages/pygad/pygad.py:1139: UserWarning: The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.
  warnings.warn("The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution a

/local/lmseifert/conda_envs/clapton/lib/python3.12/site-packages/pygad/pygad.py:1139: UserWarning: The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.
  warnings.warn("The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.")


[0, array([-2.15657864, -1.02919541, -1.12738324,  0.        ]), array([0, 2, 0, 3, 1, 0, 0, 0, 2, 2, 1, 3], dtype=object)]
[0, array([-2.16029108, -1.03290785, -1.12738324,  0.        ]), array([1, 2, 2, 3, 3, 1, 2, 2, 2, 2, 1, 2], dtype=object)]
[0, array([-4.26795418, -2.01244774, -2.25550644,  0.        ]), array([2, 0, 1, 0, 1, 2, 0, 2, 0, 1, 3, 0], dtype=object)]
[0, array([-2.17298653, -1.0456033 , -1.12738324,  0.        ]), array([1, 2, 0, 2, 2, 3, 0, 0, 2, 3, 1, 0], dtype=object)]
[1, array([-4.27694793, -2.02144149, -2.25550644,  0.        ]), array([2, 0, 3, 2, 3, 2, 3, 1, 0, 1, 1, 3], dtype=object)][1, array([-4.27559815, -2.02009171, -2.25550644,  0.        ]), array([2, 0, 3, 2, 3, 0, 3, 3, 0, 1, 3, 1], dtype=object)][1, array([-2.17364177, -1.04625854, -1.12738324,  0.        ]), array([0, 2, 0, 3, 1, 0, 0, 0, 2, 2, 2, 1], dtype=object)]


[1, array([-4.26829499, -2.01278855, -2.25550644,  0.        ]), array([3, 0, 1, 0, 2, 2, 3, 3, 2, 1, 3, 2], dtype=object)]
[2, arra

In [ ]:
# the best parameters are
ks_best

[2, 2, 1, 1, 3, 2, 0, 0, 2, 2, 2, 1]

In [ ]:
# with noisy/noiseless energy
energy_noisy, energy_noiseless

(np.float64(-2.0237951035451354), np.float64(-2.2555064377753498))

In [ ]:
# the corresponding circuit is
vqe_pcirc.assign(ks_best)
vqe_pcirc.snapshot_noiseless().circ_snapshot_noiseless.diagram()

q0: -Y------S-----X-@-I-Z---
                  | |
q1: -Y------S_DAG-|-X-@-I-Z-
                  |   |
q2: -SQRT_Y-Z-----@---X-Y-S-